#### Get mapping of dungeon -> id

In their result jsons blizz uses interger dungeon ids. Get mapping of dungeon names to ids. 

Note that these ids are for the "challenge mode" of the dungeon. There is another argumen "map" with its own id, which refers to the overall location (regardless of difficulty). We don't care about the map id.

In [15]:
import requests
import importlib
import blizzard_api
import mysql
import mplusdb
import blizzard_credentials
import pandas as pd

In [2]:
#grab access token
importlib.reload(blizzard_credentials)
tokens_file_path = '.api_tokens'
authorization = blizzard_credentials.Credentials(tokens_file_path)
access_token = authorization.access_token

In [13]:
#get m+ dungeon index
url = 'https://us.api.blizzard.com/data/wow/mythic-keystone/dungeon/index?namespace=dynamic-us&locale=en_US&access_token='
api_url = url + access_token
key_index_response = requests.get(api_url)

In [14]:
#these are ids for BFA dungeons
bfa_key_index = pd.DataFrame(key_index_response.json()['dungeons'])
bfa_key_index[['id', 'name']]

,id,name
0,244,Atal'Dazar
1,245,Freehold
2,246,Tol Dagor
3,247,The MOTHERLODE!!
4,248,Waycrest Manor
5,249,Kings' Rest
6,250,Temple of Sethraliss
7,251,The Underrot
8,252,Shrine of the Storm
9,353,Siege of Boralus


In [37]:
#there is no index call for Legion dungeons
#so here is a programmatic sweep of every id from 0 to 243

leader_board_url = ('https://us.api.blizzard.com/data/wow/'
                    'connected-realm/11/mythic-leaderboard/'
                    '{potential_id}/period/625?namespace=dynamic-us&'
                    'locale=en_US&access_token={access_token}')

valid_ids = []
for potential_id in range(0,244):
    url = leader_board_url.format(potential_id = potential_id, 
                                  access_token = access_token)
    response = requests.get(url)
    if response.status_code == 200:
        valid_ids.append(potential_id)
    print('\rtesting %s out of %s' % (potential_id, 244), end='')
print('')
print(valid_ids)

testing 243 out of 244
[197, 198, 199, 200, 206, 207, 208, 209, 210, 227, 233, 234, 239]


dungeon period
197 597
198 597
199 597
200 597
206 597
207 597
208 597
209 597
210 597
227 597
233 597
234 597
239 605


In [44]:
# Previous cell returned these valid ids
# valid_ids = [197, 198, 199, 200, 206, 207, 208, 209, 210, 227, 233, 234, 239]
legion_dungeon_id_map = []
for id_ in valid_ids:
    url = leader_board_url.format(potential_id = id_, 
                                  access_token = access_token)
    response = requests.get(url)
    dungeon_name = response.json()['map']['name']
    legion_dungeon_id_map.append([id_, dungeon_name])
print(legion_dungeon_id_map)

[[197, 'Eye of Azshara'], [198, 'Darkheart Thicket'], [199, 'Black Rook Hold'], [200, 'Halls of Valor'], [206, "Neltharion's Lair"], [207, 'Vault of the Wardens'], [208, 'Maw of Souls'], [209, 'The Arcway'], [210, 'Court of Stars'], [227, 'Return to Karazhan'], [233, 'Cathedral of Eternal Night'], [234, 'Return to Karazhan'], [239, 'Seat of the Triumvirate']]


In [47]:
legion_key_index = pd.DataFrame(legion_dungeon_id_map,
                                columns = ['id', 'name'])
legion_key_index

,id,name
0,197,Eye of Azshara
1,198,Darkheart Thicket
2,199,Black Rook Hold
3,200,Halls of Valor
4,206,Neltharion's Lair
5,207,Vault of the Wardens
6,208,Maw of Souls
7,209,The Arcway
8,210,Court of Stars
9,227,Return to Karazhan


In [50]:
#stack BFA and Legion keys together
#and push to MDB
bfa_legi_keys = pd.concat([legion_key_index, bfa_key_index[['id', 'name']]])
bfa_legi_keys

,id,name
0,197,Eye of Azshara
1,198,Darkheart Thicket
2,199,Black Rook Hold
3,200,Halls of Valor
4,206,Neltharion's Lair
5,207,Vault of the Wardens
6,208,Maw of Souls
7,209,The Arcway
8,210,Court of Stars
9,227,Return to Karazhan


In [62]:
insert_query = 'INSERT into dungeon VALUES '
values = []
for id_, name in bfa_legi_keys.values.tolist():
    value = '(%s, "%s")' % (id_, name)
    values.append(value)
values_string = ','.join(values)
insert_query += values_string
print(insert_query)

INSERT into dungeon VALUES (197, "Eye of Azshara"),(198, "Darkheart Thicket"),(199, "Black Rook Hold"),(200, "Halls of Valor"),(206, "Neltharion's Lair"),(207, "Vault of the Wardens"),(208, "Maw of Souls"),(209, "The Arcway"),(210, "Court of Stars"),(227, "Return to Karazhan"),(233, "Cathedral of Eternal Night"),(234, "Return to Karazhan"),(239, "Seat of the Triumvirate"),(244, "Atal'Dazar"),(245, "Freehold"),(246, "Tol Dagor"),(247, "The MOTHERLODE!!"),(248, "Waycrest Manor"),(249, "Kings' Rest"),(250, "Temple of Sethraliss"),(251, "The Underrot"),(252, "Shrine of the Storm"),(353, "Siege of Boralus"),(369, "Operation: Mechagon - Junkyard"),(370, "Operation: Mechagon - Workshop")


In [11]:
#Interface has changed... Plus let's not overwrite the tables.

#grab list of realm ids from MDB
#conn.close()

#mdb = mplus_db.MplusDB('.db_config')
#conn = mdb.connect()
#cursor = conn.cursor()
#cursor.execute(insert_query)
#conn.commit()
#cursor.close()
#conn.close()

------------

In [19]:
#wonderful, now lets find when each dugeon first appears on the leaderboards
# Legion
leader_board_url_ = ('https://us.api.blizzard.com/data/wow/'
                    'connected-realm/11/mythic-leaderboard/'
                    '{potential_id}/period/{period}?namespace=dynamic-us&'
                    'locale=en_US&access_token={access_token}')

print('dungeon', 'period')
valid_ids = [197, 198, 199, 200, 206, 207, 208, 209, 210, 227, 233, 234, 239]
valid_ids += [244, 245, 246, 247, 248, 249, 250, 251, 252, 353, 369, 370]

for dungeon_id in valid_ids:
    range_ = range(597, 658)
    if dungeon_id > 239:
        range_ = range(658, 762)
    for period in range_:
        url = leader_board_url_.format(potential_id = dungeon_id, 
                                      access_token = access_token,
                                      period=period)
        response = requests.get(url)
        if response.status_code == 200:
            print(dungeon_id, period)
            break

dungeon period
197 597
198 597
199 597
200 597
206 597
207 597
208 597
209 597
210 597
227 597
233 597
234 597
239 605
244 658
245 658
246 658
247 658
248 658
249 658
250 658
251 658
252 658
353 658
369 730
370 730


In [12]:
# dungeon timers
timer_in_mins = {
    244: 30,
    245: 33,
    246: 36,
    247: 39,
    248: 39,
    249: 42,
    250: 36,
    251: 33,
    252: 42,
    353: 36,
    369: 38,
    370: 32
}

timer_in_ms = dict(zip(timer_in_mins.keys(),
                       [t*60*1000 for t in timer_in_mins.values()]))

In [13]:
timer_in_ms

{244: 1800000,
 245: 1980000,
 246: 2160000,
 247: 2340000,
 248: 2340000,
 249: 2520000,
 250: 2160000,
 251: 1980000,
 252: 2520000,
 353: 2160000,
 369: 2280000,
 370: 1920000}

In [20]:
mdb = mplusdb.MplusDatabase('.db_config')
conn = mdb.connect()
cursor = conn.cursor()
for id_, time in timer_in_ms.items():
    query = 'update dungeon set timer_in_ms = %d where id = %d' % (time, id_)
    print(query)
    cursor.execute(query)
conn.commit()
cursor.close()
conn.close()

update dungeon set timer_in_ms = 1800000 where id = 244
update dungeon set timer_in_ms = 1980000 where id = 245
update dungeon set timer_in_ms = 2160000 where id = 246
update dungeon set timer_in_ms = 2340000 where id = 247
update dungeon set timer_in_ms = 2340000 where id = 248
update dungeon set timer_in_ms = 2520000 where id = 249
update dungeon set timer_in_ms = 2160000 where id = 250
update dungeon set timer_in_ms = 1980000 where id = 251
update dungeon set timer_in_ms = 2520000 where id = 252
update dungeon set timer_in_ms = 2160000 where id = 353
update dungeon set timer_in_ms = 2280000 where id = 369
update dungeon set timer_in_ms = 1920000 where id = 370
